In [467]:
from pulp import *
import math



In [477]:
NODES = ['N1','N2','N3','N4']

LINKS = [('N1','N2'), ('N1','N4'), ('N2','N3'), ('N3','N4'), ('N2','N1'), ('N3','N2'), ('N4','N1'), ('N4','N3')
        ]
#spectro
F = [ 1, 2, 3, 4, 5, 6, 7, 8 ]

#core per link
C = {('N1','N2') : [1,2,3], 
     ('N1','N4') : [1,2,3], 
     ('N2','N3') : [1,2,3], 
     ('N3','N4') : [1,2,3], 
     ('N2','N1') : [1,2,3], 
     ('N3','N2') : [1,2,3], 
     ('N4','N1') : [1,2,3], 
     ('N4','N3') : [1,2,3]
    }

demands = { 1: ('N1','N2'),
            2: ('N1','N3'),
            3: ('N1','N4'),
            4: ('N2','N4'),
          }

omega = { 1: 2,
          2: 4,
          3: 2,
          4: 4
        }

#paths per demand
pi = { 1: [ frozenset({('N1','N2')}) ],
       2: [ frozenset({('N1','N2'), ('N2','N3')}),
            frozenset({('N1','N4'), ('N4','N3')})
          ],
       3: [ frozenset({('N1','N4')}) ],
       4: [ frozenset({('N2','N1'), ('N1','N4')}),
            frozenset({('N2','N3'), ('N3','N4')})
          ] 
     }
    

#links per path
H = {   frozenset({('N1','N2')}) : [ ('N1','N2') ],
        frozenset({('N1','N2'), ('N2','N3')}) : [ ('N1','N2'), ('N2','N3') ],
        frozenset({('N1','N4'), ('N4','N3')}) : [ ('N1','N4'), ('N4','N3') ],
        frozenset({('N1','N4')}) : [ ('N1','N4') ],
        frozenset({('N2','N1'), ('N1','N4')}) : [ ('N2','N1'), ('N1','N4') ],
        frozenset({('N2','N3'), ('N3','N4')}) : [ ('N2','N3'), ('N3','N4') ],
    }

#core per path
gamma =  { frozenset({('N1','N2')}) : [ 3 ],
       frozenset({('N1','N2'), ('N2','N3')}) : [ 3, 3 ],
       frozenset({('N1','N4'), ('N4','N3')}) : [ 3, 3 ],
       frozenset({('N1','N4')}) : [ 3 ],
       frozenset({('N2','N1'), ('N1','N4')}) : [ 3, 3 ],
       frozenset({('N2','N3'), ('N3','N4')}) : [ 3, 3 ],
     }

#crosstalk threeshold
E = -30


M = sum([omega[i] for i in omega])




In [478]:
k = 3.16e-5
B = 10e7
R = 0.01
A = 4.5e-5
L = 390000000000000

In [479]:
numerador = 2 - 2 * math.exp( -3 * 2 * ( pow(k,2) / B ) * (R/A) * L )
denominador = 1 + 2 * math.exp( -3 * 2 * ( pow(k,2) / B ) * (R/A) * L )

In [480]:
Xtl = numerador / denominador
Xtl


1.9670184596217402

In [481]:




#problem variables
prob = LpProblem("Problem", LpMinimize)

X_pi_vars = {}
for path in pi:
    X_pi_vars.update(LpVariable.dicts('t',[(p,i) for i,p in enumerate(pi[path])], 0, 1, LpBinary))
    
Fd_vars = LpVariable.dicts("Fd", [(d) for d in demands], 0, None, LpInteger)

Xd_c_uv_vars = {}
for d in demands:
    Xd_c_uv_vars.update(LpVariable.dicts("Xdc_uv", [(d,core,l) for core in range(1,4) for l in LINKS], 0, 1, LpBinary))


INVERTED_OMEGA_dD_c_uv_vars = {}
for d in demands:
    INVERTED_OMEGA_dD_c_uv_vars.update(LpVariable.dicts("INVERTED_OMEGA_dD_c_uv", [(d,core,l) for core in range(1,4) for l in LINKS], 0, None, LpInteger))


Ms_vars = LpVariable.dicts("Ms", [(d,c) for d in demands for c in range(1,4)], 0, None, LpInteger)







In [482]:

#contraint 1
prob += lpSum(Ms_vars)


#contraint 2
for d in demands:
    for c in range(1,4):
        prob += Fd_vars[d] + omega[d] <= Ms_vars[d,c]
        
        
        
#contraint 3
for p in pi:
    prob += lpSum([X_pi_vars[p,i] for i,p in enumerate(pi[p])]) == 1
        

#contraint 4



#contraint 5
for d in demands:
    for p in pi: 
        if (p == d):
            for i,pp in enumerate(pi[p]):
                for g in gamma:
                    if(g == pp):
                        prob += lpSum([Xd_c_uv_vars[d,c,link] for c in range(1,4) for ii,link in enumerate(pp)]) - lpSum([gamma[pp][ii] * X_pi_vars[pp,i] for ii,link in enumerate(pp)]) == 0       
                



#contraint 6




#contraint 7


#contraint 8


#contraint 9






In [483]:
prob.solve()

1

In [484]:
# %%time

print('\nMinimum Profit:')
pulp.value(prob.objective)


Minimum Profit:


36.0

In [485]:
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

Fd_1 = 0.0
Fd_2 = 0.0
Fd_3 = 0.0
Fd_4 = 0.0
Ms_(1,_1) = 2.0
Ms_(1,_2) = 2.0
Ms_(1,_3) = 2.0
Ms_(2,_1) = 4.0
Ms_(2,_2) = 4.0
Ms_(2,_3) = 4.0
Ms_(3,_1) = 2.0
Ms_(3,_2) = 2.0
Ms_(3,_3) = 2.0
Ms_(4,_1) = 4.0
Ms_(4,_2) = 4.0
Ms_(4,_3) = 4.0
Xdc_uv_(1,_1,_('N1',_'N2')) = 1.0
Xdc_uv_(1,_2,_('N1',_'N2')) = 1.0
Xdc_uv_(1,_3,_('N1',_'N2')) = 1.0
Xdc_uv_(2,_1,_('N1',_'N2')) = 0.0
Xdc_uv_(2,_1,_('N1',_'N4')) = 1.0
Xdc_uv_(2,_1,_('N2',_'N3')) = 0.0
Xdc_uv_(2,_1,_('N4',_'N3')) = 1.0
Xdc_uv_(2,_2,_('N1',_'N2')) = 0.0
Xdc_uv_(2,_2,_('N1',_'N4')) = 1.0
Xdc_uv_(2,_2,_('N2',_'N3')) = 0.0
Xdc_uv_(2,_2,_('N4',_'N3')) = 1.0
Xdc_uv_(2,_3,_('N1',_'N2')) = 0.0
Xdc_uv_(2,_3,_('N1',_'N4')) = 1.0
Xdc_uv_(2,_3,_('N2',_'N3')) = 0.0
Xdc_uv_(2,_3,_('N4',_'N3')) = 1.0
Xdc_uv_(3,_1,_('N1',_'N4')) = 1.0
Xdc_uv_(3,_2,_('N1',_'N4')) = 1.0
Xdc_uv_(3,_3,_('N1',_'N4')) = 1.0
Xdc_uv_(4,_1,_('N1',_'N4')) = 0.0
Xdc_uv_(4,_1,_('N2',_'N1')) = 0.0
Xdc_uv_(4,_1,_('N2',_'N3')) = 1.0
Xdc_uv_(4,_1,_('N3',_'N4')) = 1.0
Xdc_uv_(4,_2,_('

Test with restrtiction (5)

In [441]:
for d in demands:
    print("ddemanda", d)
    for p in pi: 
        if (p == d):
            for i,pp in enumerate(pi[p]):
                print("caminho",i)
                for g in gamma:
                    if(g == pp):
                        print(pp)
#                         for ii,link in enumerate(pp):
#                             print(gamma[pp][ii],link,X_pi_vars_test[pp,i])
#                             for c in range(1,4):
#                                 print(c,Xd_c_uv_vars_test[d,c,link])
                        print("soma 1",sum([Xd_c_uv_vars_test[d,c,link] for c in range(1,4) for ii,link in enumerate(pp)]))
                            
                        print("soma 2",sum([gamma[pp][ii] * X_pi_vars_test[pp,i] for ii,link in enumerate(pp)]))
                        print("\n")
                        
    

ddemanda 1
caminho 0
frozenset({('N1', 'N2')})
soma 1 1
soma 2 3


ddemanda 2
caminho 0
frozenset({('N1', 'N2'), ('N2', 'N3')})
soma 1 0
soma 2 6


caminho 1
frozenset({('N1', 'N4'), ('N4', 'N3')})
soma 1 1
soma 2 0


ddemanda 3
caminho 0
frozenset({('N1', 'N4')})
soma 1 0
soma 2 0


ddemanda 4
caminho 0
frozenset({('N1', 'N4'), ('N2', 'N1')})
soma 1 0
soma 2 0


caminho 1
frozenset({('N3', 'N4'), ('N2', 'N3')})
soma 1 0
soma 2 6




In [386]:
X_pi_vars_test = {(frozenset({('N1', 'N2')}), 0): 1,
 (frozenset({('N1', 'N2'), ('N2', 'N3')}),0): 1,
 (frozenset({('N1', 'N4'), ('N4', 'N3')}),1): 0,
 (frozenset({('N1', 'N4')}), 0): 0,
 (frozenset({('N1', 'N4'), ('N2', 'N1')}),0): 0,
 (frozenset({('N2', 'N3'), ('N3', 'N4')}),1): 1}

In [383]:
X_pi_vars_test

{(frozenset({('N1', 'N2')}), 0): 1,
 (frozenset({('N1', 'N2'), ('N2', 'N3')}), 0): 1,
 (frozenset({('N1', 'N4'), ('N4', 'N3')}), 1): 0,
 (frozenset({('N1', 'N4')}), 0): 0,
 (frozenset({('N1', 'N4'), ('N2', 'N1')}), 0): 1,
 (frozenset({('N2', 'N3'), ('N3', 'N4')}), 1): 1}

In [440]:
Xd_c_uv_vars_test = {
 (1, 1, ('N1', 'N2')): 1,
 (1, 1, ('N1', 'N4')): 0,
 (1, 1, ('N2', 'N3')): 0,
 (1, 1, ('N3', 'N4')): 0,
 (1, 1, ('N2', 'N1')): 0,
 (1, 1, ('N3', 'N2')): 0,
 (1, 1, ('N4', 'N1')):0,
 (1, 1, ('N4', 'N3')): 0,
 (1, 2, ('N1', 'N2')): 0,
 (1, 2, ('N1', 'N4')): 0,
 (1, 2, ('N2', 'N3')): 0,
 (1, 2, ('N3', 'N4')): 0,
 (1, 2, ('N2', 'N1')): 0,
 (1, 2, ('N3', 'N2')): 0,
 (1, 2, ('N4', 'N1')): 0,
 (1, 2, ('N4', 'N3')): 0,
 (1, 3, ('N1', 'N2')): 0,
 (1, 3, ('N1', 'N4')): 0,
 (1, 3, ('N2', 'N3')): 0,
 (1, 3, ('N3', 'N4')): 0,
 (1, 3, ('N2', 'N1')): 0,
 (1, 3, ('N3', 'N2')): 0,
 (1, 3, ('N4', 'N1')): 0,
 (1, 3, ('N4', 'N3')): 0,
 (2, 1, ('N1', 'N2')): 0,
 (2, 1, ('N1', 'N4')): 1,
 (2, 1, ('N2', 'N3')): 0,
 (2, 1, ('N3', 'N4')): 0,
 (2, 1, ('N2', 'N1')): 0,
 (2, 1, ('N3', 'N2')): 0,
 (2, 1, ('N4', 'N1')): 0,
 (2, 1, ('N4', 'N3')): 0,
 (2, 2, ('N1', 'N2')): 0,
 (2, 2, ('N1', 'N4')): 0,
 (2, 2, ('N2', 'N3')): 0,
 (2, 2, ('N3', 'N4')): 0,
 (2, 2, ('N2', 'N1')): 0,
 (2, 2, ('N3', 'N2')): 0,
 (2, 2, ('N4', 'N1')): 0,
 (2, 2, ('N4', 'N3')): 0,
 (2, 3, ('N1', 'N2')): 0,
 (2, 3, ('N1', 'N4')): 0,
 (2, 3, ('N2', 'N3')): 0,
 (2, 3, ('N3', 'N4')): 0,
 (2, 3, ('N2', 'N1')): 0,
 (2, 3, ('N3', 'N2')): 0,
 (2, 3, ('N4', 'N1')): 0,
 (2, 3, ('N4', 'N3')): 0,
 (3, 1, ('N1', 'N2')): 0,
 (3, 1, ('N1', 'N4')): 0,
 (3, 1, ('N2', 'N3')): 0,
 (3, 1, ('N3', 'N4')): 0,
 (3, 1, ('N2', 'N1')): 0,
 (3, 1, ('N3', 'N2')): 0,
 (3, 1, ('N4', 'N1')): 0,
 (3, 1, ('N4', 'N3')): 0,
 (3, 2, ('N1', 'N2')): 0,
 (3, 2, ('N1', 'N4')): 0,
 (3, 2, ('N2', 'N3')): 0,
 (3, 2, ('N3', 'N4')): 0,
 (3, 2, ('N2', 'N1')): 0,
 (3, 2, ('N3', 'N2')):0,
 (3, 2, ('N4', 'N1')): 0,
 (3, 2, ('N4', 'N3')): 0,
 (3, 3, ('N1', 'N2')): 0,
 (3, 3, ('N1', 'N4')): 0,
 (3, 3, ('N2', 'N3')): 0,
 (3, 3, ('N3', 'N4')): 0,
 (3, 3, ('N2', 'N1')): 0,
 (3, 3, ('N3', 'N2')): 0,
 (3, 3, ('N4', 'N1')): 0,
 (3, 3, ('N4', 'N3')):0,
 (4, 1, ('N1', 'N2')): 0,
 (4, 1, ('N1', 'N4')): 0,
 (4, 1, ('N2', 'N3')): 0,
 (4, 1, ('N3', 'N4')): 0,
 (4, 1, ('N2', 'N1')): 0,
 (4, 1, ('N3', 'N2')): 0,
 (4, 1, ('N4', 'N1')): 0,
 (4, 1, ('N4', 'N3')): 0,
 (4, 2, ('N1', 'N2')): 0,
 (4, 2, ('N1', 'N4')): 0,
 (4, 2, ('N2', 'N3')): 0,
 (4, 2, ('N3', 'N4')): 0,
 (4, 2, ('N2', 'N1')): 0,
 (4, 2, ('N3', 'N2')): 0,
 (4, 2, ('N4', 'N1')): 0,
 (4, 2, ('N4', 'N3')): 0,
 (4, 3, ('N1', 'N2')): 0,
 (4, 3, ('N1', 'N4')): 0,
 (4, 3, ('N2', 'N3')): 0,
 (4, 3, ('N3', 'N4')): 0,
 (4, 3, ('N2', 'N1')): 0,
 (4, 3, ('N3', 'N2')): 0,
 (4, 3, ('N4', 'N1')): 0,
 (4, 3, ('N4', 'N3')): 0}